In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import ray
ray.shutdown()

import joblib
from ray.util.joblib import register_ray

register_ray()
ray.init(address="127.0.0.1:6379")

import mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5001')
# mlflow.autolog() #log_models=False, exclusive=True)
import sklearn
mlflow.sklearn.autolog()#log_models=True)

digits = load_digits()

param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=5, n_iter=50, verbose=10)

with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)
